In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.3 MB/s eta 0:00:00


In [4]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296189 sha256=635af14200ac3d0309f0c43b29bfa37da25b1d7d5a859aa3685fc31ebcbbea68
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [5]:
!pip install sentence-transformers

In [10]:
%%writefile app.py
import os
import streamlit as st
import numpy as np
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import fasttext
import fasttext.util
import pdfplumber
from sentence_transformers import SentenceTransformer

# Initialize models
@st.cache_resource
def load_models():
    """Load all models with caching to improve performance"""
    models = {}
    try:
        # Load FastText
        fasttext.util.download_model('en', if_exists='ignore')
        models['fasttext'] = fasttext.load_model('cc.en.300.bin')

        # Load Sentence-BERT
        models['sbert'] = SentenceTransformer('all-MiniLM-L6-v2')

        return models
    except Exception as e:
        st.error(f"Error loading models: {str(e)}")
        return None

# Load models
models = load_models()

# Function to extract text from PDF files
def extract_text_from_pdf(pdf_file):
    text = ""
    try:
        with pdfplumber.open(pdf_file) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    except Exception as e:
        st.error(f"Error reading PDF: {e}")
    return text

# Function to calculate similarity using TF-IDF
def calculate_similarity_tfidf(job_description, resumes):
    """Calculate cosine similarity using TF-IDF vectorizer."""
    vectorizer = TfidfVectorizer()
    corpus = [job_description] + resumes
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return similarities[0]

# Function for FastText embeddings
def get_fasttext_embedding(sentence, model):
    words = sentence.lower().split()
    word_embeddings = []
    for word in words:
        try:
            word_embeddings.append(model.get_word_vector(word))
        except:
            continue
    if word_embeddings:
        return np.mean(word_embeddings, axis=0)
    return np.zeros(model.get_dimension())

# Function to calculate similarity using FastText
def calculate_similarity_fasttext(job_description, resumes):
    """Calculate cosine similarity using FastText."""
    job_description_embedding = get_fasttext_embedding(job_description, models['fasttext'])
    resume_embeddings = [get_fasttext_embedding(resume, models['fasttext']) for resume in resumes]
    similarities = [cosine_similarity([job_description_embedding], [resume_embedding])[0][0]
                   for resume_embedding in resume_embeddings]
    return similarities

# Function to calculate similarity using Sentence-BERT
def calculate_similarity_sbert(job_description, resumes):
    """Calculate cosine similarity using Sentence-BERT."""
    # Encode job description
    job_embedding = models['sbert'].encode(job_description)

    # Encode all resumes
    resume_embeddings = models['sbert'].encode(resumes)

    # Calculate similarities
    similarities = cosine_similarity([job_embedding], resume_embeddings)[0]
    return similarities

# Function to display ranked resumes
def display_ranked_resumes(similarities, resume_files):
    """Display the top 5 resumes based on similarity scores."""
    ranked_resumes = sorted(zip(resume_files, similarities), key=lambda x: x[1], reverse=True)

    st.write("### Top 5 Resumes Based on Similarity:")
    for idx, (resume_file, score) in enumerate(ranked_resumes[:5]):
        st.write(f"{idx+1}. Resume: {resume_file.name} - Similarity Score: {score:.4f}")

# Streamlit app interface
def main():
    st.title("📈 Resume Ranking Application")
    st.write("Upload resumes in PDF format, and the application will rank them based on similarity to the job description.")

    with st.spinner('Loading models... This might take a few minutes on first run.'):
        if not models:
            st.error("Error loading models. Please refresh the page.")
            return

    # Upload job description and resume files
    job_description = st.text_area("Enter the Job Description:")
    resume_files = st.file_uploader("Upload Resumes (PDF format)", accept_multiple_files=True)

    # Select the model for calculating similarity
    model_option = st.radio(
        "Select the model for similarity calculation:",
        ("TF-IDF", "FastText", "Sentence-BERT")
    )

    if st.button("Rank Resumes"):
        if job_description and resume_files:
            with st.spinner('Processing resumes...'):
                # Extract text from uploaded resumes
                resumes = [extract_text_from_pdf(resume) for resume in resume_files]

                # Calculate similarities based on selected model
                if model_option == "TF-IDF":
                    similarities = calculate_similarity_tfidf(job_description, resumes)
                elif model_option == "FastText":
                    similarities = calculate_similarity_fasttext(job_description, resumes)
                else:  # Sentence-BERT
                    similarities = calculate_similarity_sbert(job_description, resumes)

                # Display ranked resumes
                display_ranked_resumes(similarities, resume_files)
        else:
            st.warning("Please enter the job description and upload at least one resume.")

if __name__ == "__main__":
    main()


Writing app.py


In [11]:
!wget -q -O - ipv4.icanhazip.com

34.126.137.163


In [12]:
!npm install localtunnel

⠙⠹
up to date in 467ms
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [13]:
! streamlit run app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.126.137.163:8501

your url is: https://fast-pets-stick.loca.lt
2024-12-05 17:51:27.688316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 17:51:27.711381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 17:51:27.718879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 17:51:27.735917: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU inst